In [0]:
from google.colab import drive
!mkdir /content/data
drive.mount("/content/data")

mkdir: cannot create directory ‘/content/data’: File exists
Drive already mounted at /content/data; to attempt to forcibly remount, call drive.mount("/content/data", force_remount=True).
mkdir: cannot create directory ‘/content/data’: File exists
Drive already mounted at /content/data; to attempt to forcibly remount, call drive.mount("/content/data", force_remount=True).


In [0]:
import tensorflow as tf
import keras.backend as K
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Layer
import pickle
!rm -r Graph
!rm -r GraphODE

Using TensorFlow backend.


rm: cannot remove 'GraphODE': No such file or directory


In [0]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://b57a047e.ngrok.io


In [0]:
def set_gpu_config(device = "0",fraction=0.25):
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = fraction
    config.gpu_options.visible_device_list = device
    K.set_session(tf.Session(config=config))


class ODEBlock(Layer):

    def __init__(self, filters, kernel_size, **kwargs):
        self.filters = filters
        self.kernel_size = kernel_size
        super(ODEBlock, self).__init__(**kwargs)

    def build(self, input_shape):
        self.conv2d_w1 = self.add_weight("conv2d_w1", self.kernel_size + (self.filters + 1, self.filters), initializer='glorot_uniform')
        self.conv2d_w2 = self.add_weight("conv2d_w2", self.kernel_size + (self.filters + 1, self.filters), initializer='glorot_uniform')
        self.conv2d_b1 = self.add_weight("conv2d_b1", (self.filters,), initializer='zero')
        self.conv2d_b2 = self.add_weight("conv2d_b2", (self.filters,), initializer='zero')
        super(ODEBlock, self).build(input_shape)

    def call(self, x):
        t = K.constant([0, 1], dtype="float32")
        return tf.contrib.integrate.odeint(self.ode_func, x, t, rtol=1e-3, atol=1e-3)[1]

    def compute_output_shape(self, input_shape):
        return input_shape

    def ode_func(self, x, t):
        y = self.concat_t(x, t)
        y = K.conv2d(y, self.conv2d_w1, padding="same")
        y = K.bias_add(y, self.conv2d_b1)
        y = K.relu(y)

        y = self.concat_t(y, t)
        y = K.conv2d(y, self.conv2d_w2, padding="same")
        y = K.bias_add(y, self.conv2d_b2)
        y = K.relu(y)

        return y

    def concat_t(self, x, t):
        new_shape = tf.concat(
            [
                tf.shape(x)[:-1],
                tf.constant([1],dtype="int32",shape=(1,))
            ], axis=0)

        t = tf.ones(shape=new_shape) * tf.reshape(t, (1, 1, 1, 1))
        return tf.concat([x, t], axis=-1)


def build_model(input_shape, num_classes):
    x = Input(input_shape)
    y = Conv2D(64, (3, 3), activation='relu')(x)
    y = MaxPooling2D((2,2))(y)
    y = Conv2D(128, (3, 3), activation='relu')(y)
    y = MaxPooling2D((2,2))(y)
    y = ODEBlock(128, (3, 3))(y)
    y = Flatten()(y)
    y = Dense(num_classes, activation='softmax')(y)
    return Model(x,y)

In [0]:
batch_size = 32
num_classes = 151
epochs = 30
image_shape = (64,64, 1)

pickledDataset = open("/content/data/My Drive/Minor Project/toSend/faces96dataWithImgSalMBD.pickle","rb")
data = pickle.load(pickledDataset)

(x_train, y_train), (x_test, y_test) = data['res']
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

x_train = x_train.reshape((-1,) + image_shape)
x_test = x_test.reshape((-1,) + image_shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes) 


In [0]:
model = build_model(image_shape, num_classes)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

# tbCallBack = keras.callbacks.TensorBoard(log_dir='./GraphODE', histogram_freq=0, write_graph=True, write_images=True)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),callbacks=[TensorBoardColabCallback(tbc)])

model.save_weights('/content/data/My Drive/Minor Project/toSend/OdeNetF96colab_with.h5')

model.save('/content/data/My Drive/Minor Project/toSend/saved_models/OdeNetF96colab.h5')
model.save('/content/data/My Drive/Minor Project/toSend/saved_models/OdeNetF96colab.model')

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.cast instead.
Train on 2567 samples, validate on 453 samples
Epoch 1/30
2567/2567 [==============================] - 14s 5ms/step - loss: 4.0226 - acc: 0.2108 - val_loss: 1.2979 - val_acc: 0.7152
Epoch 2/30
2567/2567 [==============================] - 9s 4ms/step - loss: 0.4859 - acc: 0.8862 - val_loss: 0.1193 - val_acc: 0.9735
Epoch 3/30
2567/2567 [==============================] - 9s 4ms/step - loss: 0.0509 - acc: 0.9868 - val_loss: 0.0132 - val_acc: 1.0000
Epoch 4/30
2567/2567 [==============================] - 9s 4ms/step - loss: 0.0134 - acc: 0.9961 - val_loss: 0.0138 